In [40]:
import pandas as pd
import numpy as np
import re

In [42]:
def clean_rank_score(text):
    """ Get the pair number from each cell of rankings
        If the cell is blank, return nan
    """
    try:
        return 'pair ' + re.search(r"\d+", text).group(0)
    except:
        return np.nan
    
def apply_to_table(func, df, name_condition, mode = 'in'):
    """ Apply a function to every column of a dataset which
        matches the given
        
        If mode = 'in' (default),
        the condition is a column's name must contain name_condition
        
        If mode = 'equal',
        the condition is a column's name must equal to name_condition
        
        If mode = 'not in',
        the condition is a column's name must not contain name_condition
        
        If mode = 'not equal',
        the condition is a column's name must not equal to name_condition
    """
    new = df.copy()
    
    if mode == 'in':
        selected_cols = [col for col in df if name_condition in col]
    elif mode == 'equal':
        selected_cols = [col for col in df if col == name_condition]
    elif mode == 'not in':
        selected_cols = [col for col in df if name_condition not in col]
    elif mode == 'not equal':
        selected_cols = [col for col in df if col != name_condition]
    else:
        return "Error: mode is not defined"
    
    for col in new.columns:
        if col in selected_cols:
            new[col] = df[col].apply(func)
    return new

In [5]:
rooms = pd.read_excel('final_flex_no_ranked.xlsx', index_col='ID')
rooms.head()

,1,2,3,4
ID,,,,
6230124521,0,0,1,0
6030274221,0,1,0,0
6031779021,1,0,0,0
6030650521,0,1,0,0
6131791621,1,0,0,0


In [44]:
ranks = pd.read_csv('Data/raw_example.csv').fillna(0)
ranks.drop_duplicates(subset='เลขประจำตัวนิสิต', keep = 'last', inplace = True)
ranks.head(2)

,ประทับเวลา,ที่อยู่อีเมล,เบอร์โทร (0XX-XXX-XXXX),เลขประจำตัวนิสิต,ชื่อ-นามสกุล,Pair ที่สนใจลำดับที่ 1,Pair ที่สนใจลำดับที่ 2,Pair ที่สนใจลำดับที่ 3,Pair ที่สนใจลำดับที่ 4,Pair ที่สนใจลำดับที่ 5,Pair ที่สนใจลำดับที่ 6,Pair ที่สนใจลำดับที่ 7,Pair ที่สนใจลำดับที่ 8,Pair ที่สนใจลำดับที่ 9,Pair ที่สนใจลำดับที่ 10
0,"27/9/2020, 16:25:39",pao.chew@hotmail.com,098-016-6167,6230124521,ฐิติ ชิวชรัตน์,"Pair 17 : พี่ใหม่ ปริมล กาญจนจารี, พี่วิน วิธว...","Pair 2 : พี่ก้อง พณชิต กิตติปัญญางาม, พี่ต้น ว...","Pair 1 : พี่เล้ง ศิริวัฒน์ วงศ์จารุกร, พี่โทนี...","Pair 5 : พี่เก่ง ดร.ธิติ วัชรสินธพชัย, พี่วี ว...","Pair 15 : พี่อ้อ รัชดา ด่านพงษ์เจริญ, พี่โอห์ม...","Pair 18 : พี่ต้อง ดร. อโณทัย รัตนกุล, พี่ต้า ด...","Pair 10 : พี่เห่า ดร.วิบูลย์ อรุณธเนศ, พี่แบงก...","Pair 12 : พี่เส เสนธิป ศรีไพพรรณ, พี่แจ็ค ชวิน...","Pair 8 : พี่ประทิต ประทิต โลหะธีรภาพ, พี่ตั้ม ...","Pair 6 : พี่ถิ ถิรพันธุ์ สรรพกิจ, พี่ปันเจ ปัญ..."
2,"28/9/2020, 11:18:32",wsasijulaka@gmail.com,099-193-9153,6031843521,วรกร ศศิจุลกะ,"Pair 16 : พี่หลุยส์ ธีรพจน์ จรสโรจน์กุล, พี่โบ...","Pair 8 : พี่ประทิต ประทิต โลหะธีรภาพ, พี่ตั้ม ...","Pair 3 : พี่โบ้ พิเชษฐ สิทธิอำนวย, พี่เหมย ศิร...","Pair 17 : พี่ใหม่ ปริมล กาญจนจารี, พี่วิน วิธว...","Pair 7 : พี่โหน่ง พิสิฐ ยิ่งมโนกิจ, พี่เอ็ม ธน...","Pair 10 : พี่เห่า ดร.วิบูลย์ อรุณธเนศ, พี่แบงก...","Pair 13 : พี่โน อโณทัย อดุลพันธุ์, พี่หวัง กิต...","Pair 6 : พี่ถิ ถิรพันธุ์ สรรพกิจ, พี่ปันเจ ปัญ...","Pair 12 : พี่เส เสนธิป ศรีไพพรรณ, พี่แจ็ค ชวิน...","Pair 18 : พี่ต้อง ดร. อโณทัย รัตนกุล, พี่ต้า ด..."


In [45]:
ranks_cols = ['เลขประจำตัวนิสิต'] + [col for col in ranks.columns if 'Pair' in col]
ranks = ranks[ranks_cols]
ranks_new_cols = ['ID'] + [' '.join(['rank', str(i+1)]) for i in range(10)]
ranks.columns = ranks_new_cols
ranks.head()

# Extract numbers
rank_cleaned = apply_to_table(clean_rank_score, ranks, 'rank', 'in')

In [47]:
ROOMS = {
    'RoomA': [7,16,4,8],
    'RoomB': [9,10,14,1,3],
    'RoomC': [12,17,13,2,18],
    'RoomD': [6,11,5,15]
}

groups = dict()
for group in ROOMS.keys():
    for room in ROOMS[group]:
        groups[room] = group
#groups

In [58]:
tmp = rooms.unstack().reset_index()
tmp.columns = ['Room', 'ID', 'Value']
tmp = tmp[tmp['Value'] == 1].reset_index(drop = True).drop(['Value'], axis = 1)
group = tmp[['ID', 'Room']]
group.head()

,ID,Room
0,6031779021,1
1,6131791621,1
2,6130549021,1
3,6130461321,1
4,6131752121,1


In [59]:
rank4 = rank_cleaned[['ID', 'rank 1', 'rank 2', 'rank 3', 'rank 4']]
rank4.head()

,ID,rank 1,rank 2,rank 3,rank 4
0,6230124521,pair 17,pair 2,pair 1,pair 5
2,6031843521,pair 16,pair 8,pair 3,pair 17
3,6030274221,pair 3,pair 15,pair 12,pair 18
4,6031779021,pair 8,pair 3,pair 12,pair 1
5,6030826721,pair 2,pair 13,pair 17,pair 3


In [60]:
rank4['room'] = np.nan
for candidate in rank4['ID']:
    room = group[group['ID'] == candidate]['Room']
    room_map = {1: 'A', 2: 'B', 3: 'C', 4: 'D'}
    rank4[rank4['ID'] == candidate]['room'] = room_map[room]

/Users/pasin/opt/anaconda3/envs/env01/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


TypeError: 'Series' objects are mutable, thus they cannot be hashed